In [4]:
import pandas as pd
import math as math
import numpy as np
import csv

## Path to raw orderbook data from Binance (Bitcoin & Ethereum)
# orderbook_path = "../../data/raw/orderbook/BTC-daily-orderbook.csv"
orderbook_path = "../../data/raw/orderbook/ETH-daily-orderbook.csv"

## Path to where processed data is stored
# output_path = "../../data/processed/orderbook/BTC-1s.csv"
output_path = "../../data/processed/orderbook/ETH-1s.csv"

## Read data in chunks 
df = pd.read_csv(orderbook_path, chunksize=100000)

In [5]:
def average(a, b):
    result = (a+b)/2
    return result

def calc_spread(a, b):
    return abs(a-b)

## Process a single chunk of data
## Returns: aggregated chunk
def aggregate(chunk):
    aggregated_chunk = pd.DataFrame({'timestamp':[],'midpoint':[], 'spread':[]})
    
    timestamps = chunk['timestamp']
    
    ## convert timestamps to dictionary since there are multiple records with the same timestamp
    timestamps = list(dict.fromkeys(timestamps))
    
    ## filters
    filt2 = chunk['side'] == 'a'
    filt3 = chunk['side'] == 'b'
    filt4 = chunk['qty'] > 0
    
    idx = 0
    asks = []
    bids = []
    
    begin_seconds_timestamp = 0
    previous_midpoint = 0
    previous_spread = 0
    ## loop over the timestamps in the chunk
    while(idx < len(timestamps)):
        
        ## get all the records with the current timestamp
        filt1 = chunk['timestamp'] == timestamps[idx]
        timestamp = timestamps[idx]
        
        if begin_seconds_timestamp == 0:
            begin_seconds_timestamp = timestamp
        
        if timestamp > (begin_seconds_timestamp + 1000):
            if not asks or not bids:
                midpoint = previous_midpoint
                spread = previous_spread
            else:
                midpoint = average(min(asks), max(bids))
                spread = calc_spread(min(asks), max(bids))
            temp = pd.DataFrame({'timestamp':[timestamp],'midpoint':[midpoint], 'spread':[spread]})
            aggregated_chunk = pd.concat([aggregated_chunk,temp])
            asks = []
            bids = []
            begin_seconds_timestamp = timestamp
            previous_midpoint = midpoint
            previous_spread = spread
        
        ## filter out the lowest ask and the highest bid among all the asks and bids
        ## linked to the current timestamp
        lowest_ask = chunk.loc[filt1 & filt2 & filt4]['price'].min() 
        highest_bid = chunk.loc[filt1 & filt3 & filt4]['price'].max()
                
        if not math.isnan(lowest_ask):
            asks.append(lowest_ask)
        if not math.isnan(highest_bid):
            bids.append(highest_bid)
                    
        idx = idx + 1
       
    return aggregated_chunk        

In [6]:
output = pd.DataFrame({'timestamp':[], 'midpoint':[], 'spread':[]})

## Output 
output.to_csv(output_path, mode='a', index=False)

## Process every chunk in the data frame
count = 0
for chunk in df:
    
    ## Remove head and tail timestamps of each chunk to avoid overlap 
    head = chunk.head(1)['timestamp'].values[0]
    tail = chunk.tail(1)['timestamp'].values[0]
    
    filter1 = (chunk['timestamp'] != head)
    filter2 = (chunk['timestamp'] != tail)
    chunk = chunk.loc[filter1 & filter2]
    output_chunk = aggregate(chunk)
    
    output_chunk['timestamp'] = output_chunk['timestamp'].astype(np.int64)
    output_chunk.to_csv(output_path, mode='a',header=False, index=False, float_format='%.3f')
    
    print(f"Processed chunk {count}")
    count = count + 1
    
print("Finished processing")

Processed chunk 0
Processed chunk 1
Processed chunk 2
Processed chunk 3
Processed chunk 4
Processed chunk 5
Processed chunk 6
Processed chunk 7
Processed chunk 8
Processed chunk 9
Processed chunk 10
Processed chunk 11
Processed chunk 12
Processed chunk 13
Processed chunk 14
Processed chunk 15
Processed chunk 16
Processed chunk 17
Processed chunk 18
Processed chunk 19
Processed chunk 20
Processed chunk 21
Processed chunk 22
Processed chunk 23
Processed chunk 24
Processed chunk 25
Processed chunk 26
Processed chunk 27
Processed chunk 28
Processed chunk 29
Processed chunk 30
Processed chunk 31
Processed chunk 32
Processed chunk 33
Processed chunk 34
Processed chunk 35
Processed chunk 36
Processed chunk 37
Processed chunk 38
Processed chunk 39
Processed chunk 40
Processed chunk 41
Processed chunk 42
Processed chunk 43
Processed chunk 44
Processed chunk 45
Processed chunk 46
Processed chunk 47
Processed chunk 48
Processed chunk 49
Processed chunk 50
Processed chunk 51
Processed chunk 52
Pro

Processed chunk 416
Processed chunk 417
Processed chunk 418
Processed chunk 419
Processed chunk 420
Processed chunk 421
Processed chunk 422
Processed chunk 423
Processed chunk 424
Processed chunk 425
Processed chunk 426
Processed chunk 427
Processed chunk 428
Processed chunk 429
Processed chunk 430
Processed chunk 431
Processed chunk 432
Finished processing
